In [33]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt
from matplotlib import colors as mcolors
from sklearn.metrics import roc_curve
from scipy import interpolate
from sklearn.utils import shuffle

In [34]:
#Reading files into a log
import sys
import logging

nblog = open("nb.log", "a+")
sys.stdout.echo = nblog
sys.stderr.echo = nblog

get_ipython().log.handlers[0].stream = nblog
get_ipython().log.setLevel(logging.INFO)

%autosave 5

Autosaving every 5 seconds


In [10]:
os.chdir(r'/home/manhducnmd/pp_dijet/Results_full')

In [11]:
np.random.seed(42)

In [12]:
tf.random.set_seed(42)

In [6]:
def CWoLA(p_i):
    
    input_1 = keras.layers.Input(shape = (p_i,p_i,1), name = "jet_1")
    y = keras.layers.BatchNormalization()(input_1)
    input_2 = keras.layers.Input(shape = (p_i,p_i,1), name = "jet_2")
    z = keras.layers.BatchNormalization()(input_2)
    
    x = keras.layers.Concatenate(axis = 0)([y, z])

    x = keras.layers.Conv2D(64, (5,5), padding = 'same')(input_1)
    x = keras.layers.MaxPool2D(pool_size = (2,2))(x)
        
    x = keras.layers.Conv2D(64, (5, 5), padding='same')(x)
    x = keras.layers.MaxPool2D(pool_size = (2,2))(x)

    x = keras.layers.Conv2D(128, (3,3), padding='same')(x)
    x = keras.layers.MaxPool2D(pool_size = (2,2))(x)
    
    x = keras.layers.Conv2D(128, (3,3))(x)
    x = keras.layers.Flatten()(x)
    
    x = keras.layers.Dense(128, activation = 'relu')(x)
    x = keras.layers.Dense(128, activation = 'relu')(x)
    x = keras.layers.Dense(128, activation = 'relu')(x)
    
    x = keras.layers.Dense(1, activation = 'sigmoid')(x)
    
    
    return keras.Model(         
    inputs=[input_1, input_2],
    outputs=x,)

In [ ]:
n_train_sr_bkg = 25000
rng = np.random.default_rng(seed=42)

mean_results = []
std_results = []
for p_1 in [25]:
    bkg_sr_jet_1 = np.load(f'background_images_sr_{p_1}_jet_1.npy')
    bkg_sr_jet_2 = np.load(f'background_images_sr_{p_1}_jet_2.npy')
    
    bkg_sb_jet_1 = np.load(f'background_images_sb_{p_1}_jet_1.npy')
    bkg_sb_jet_2 = np.load(f'background_images_sb_{p_1}_jet_2.npy')
    
    sb_sr_bkg = np.shape(bkg_sb_jet_1)[0]/np.shape(bkg_sr_jet_2)[0]
    n_train_sb_bkg = int(np.round(n_train_sr_bkg*sb_sr_bkg))
    n_test_sr_bkg = 20000
    #First jet, background, SR
    bkg_train_sr_1 = bkg_sr_jet_1[0:n_train_sr_bkg]
    bkg_test_1 = bkg_sr_jet_1[n_train_sr_bkg:n_train_sr_bkg+n_test_sr_bkg]
    #Second jet, background, SR
    bkg_train_sr_2 = bkg_sr_jet_2[0:n_train_sr_bkg]
    bkg_test_2 = bkg_sr_jet_2[n_train_sr_bkg:n_train_sr_bkg+n_test_sr_bkg]
    #First jet, background, SB
    bkg_train_sb_1 = bkg_sb_jet_1[0:n_train_sb_bkg]
    #Second jet, background, SB
    bkg_train_sb_2 = bkg_sb_jet_2[0:n_train_sb_bkg]
    
    signal_sr_jet_1 = np.load(f'id10_sr_{p_1}_jet_1.npy')
    signal_sr_jet_2 = np.load(f'id10_sr_{p_1}_jet_2.npy')
    
    signal_sb_jet_1 = np.load(f'id10_sb_{p_1}_jet_1.npy')
    signal_sb_jet_2 = np.load(f'id10_sb_{p_1}_jet_2.npy')
    
    sb_sr_signal = np.shape(signal_sb_jet_1)[0]/np.shape(signal_sr_jet_1)[0]
    for n_train_sr_signal in [50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 600, 700, 800, 900, 1000, 1100, 1200]:
        significance = np.sqrt(2*((n_train_sr_signal+25000)*np.log(n_train_sr_signal/25000 + 1) - n_train_sr_signal))
        
        n_train_sb_signal = int(np.round(n_train_sr_signal*sb_sr_signal))
        n_test_sr_signal = 20000

        #First jet, signal, SR
        signal_train_sr_1 = signal_sr_jet_1[0:n_train_sr_signal]
        signal_test_1 = signal_sr_jet_1[n_train_sr_signal:n_train_sr_signal+n_test_sr_signal]

        #Second jet, signal, SR
        signal_train_sr_2 = signal_sr_jet_2[0:n_train_sr_signal]
        signal_test_2 = signal_sr_jet_2[n_train_sr_signal:n_train_sr_signal+n_test_sr_signal]

        #First jet, signal, SB
        signal_train_sb_1 = signal_sb_jet_1[0:n_train_sb_signal]

        #Second jet, signal, SB
        signal_train_sb_2 = signal_sb_jet_2[0:n_train_sb_signal]
        
        train_sr_1 = np.concatenate((signal_train_sr_1, bkg_train_sr_1))
        train_sr_2 = np.concatenate((signal_train_sr_2, bkg_train_sr_2))

        train_label_sr = np.ones(np.shape(train_sr_1)[0])
        
        train_sb_1 = np.concatenate((signal_train_sb_1, bkg_train_sb_1))
        train_sb_2 = np.concatenate((signal_train_sb_2, bkg_train_sb_2))

        train_label_sb = np.zeros(np.shape(train_sb_1)[0])
        
        test_label_signal = np.ones(np.shape(signal_test_1)[0])
        test_label_bkg = np.zeros(np.shape(bkg_test_1)[0])
        
        x_train_1 = np.concatenate((train_sr_1, train_sb_1))
        x_train_2 = np.concatenate((train_sr_2, train_sb_2))
        y_train = np.concatenate((train_label_sr, train_label_sb))
        
        indices = rng.permutation(np.shape(y_train)[0])
        
        x_train_1 = x_train_1[indices]
        x_train_2 = x_train_2[indices]
        y_train = y_train[indices]
        
        x_test_1 = np.concatenate((signal_test_1, bkg_test_1))
        x_test_2 = np.concatenate((signal_test_2, bkg_test_2))
        y_test = np.concatenate((test_label_signal, test_label_bkg))
        
        x_train_1 = x_train_1.reshape((np.shape(x_train_1)[0], p_1, p_1, 1))
        x_train_2 = x_train_2.reshape((np.shape(x_train_2)[0], p_1, p_1, 1))
    
        x_test_1 = x_test_1.reshape((np.shape(x_test_1)[0],p_1,p_1,1))
        x_test_2 = x_test_2.reshape((np.shape(x_test_2)[0],p_1,p_1,1))
        epsilon_results = []
        for i in range(10):
            print(f'Training with significance = {significance:.2f}, run {i}')
            cwola = CWoLA(p_1)
            loss_object = keras.losses.BinaryCrossentropy()
            optimizer = keras.optimizers.Adam(learning_rate = 1e-4)
            cwola.compile(loss = loss_object, optimizer = optimizer, metrics = ['accuracy'])
            early_stopping = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience=30, restore_best_weights=True)
            cwola.fit({'jet_1': x_train_1, 'jet_2': x_train_2}, y_train, validation_split = 0.2,
              shuffle = True, batch_size = 500, callbacks = [early_stopping], epochs = 1000)  
            x_predict = cwola.predict([x_test_1, x_test_2])
            fpr, tpr, th = roc_curve(y_test, x_predict)
            f = interpolate.interp1d(fpr, tpr)
            epsilon_s = f([0.001, 0.01, 0.1])
            print(epsilon_s)
            epsilon_results.append(epsilon_s)
            
        epsilon_results = np.array(epsilon_results)
        print(f'Mean: of significance = {significance:.2f}', np.mean(epsilon_results, axis = 0))
        print(f'Std of significance = {significance:.2f}: ', np.std(epsilon_results, axis = 0))
        mean_results.append(np.mean(epsilon_results, axis = 0))
        std_results.append(np.std(epsilon_results, axis = 0))

mean_results = np.array(mean_results)
std_results = np.array(std_results)
os.chdir(r'/home/manhducnmd/pp_dijet/Results_full')
with open('cwola_mean_id10.npy','wb') as f:
    np.save(f, mean_results)
with open('cwola_std_id10.npy', 'wb') as g:
    np.save(g, std_results)
    

Training with significance = 0.32, run 0
Train on 40762 samples, validate on 10191 samples
Epoch 1/1000
40762/40762 [==============================] - 2s 45us/sample - loss: 0.7069 - accuracy: 0.4988 - val_loss: 0.6940 - val_accuracy: 0.5012
Epoch 2/1000
40762/40762 [==============================] - 1s 30us/sample - loss: 0.6968 - accuracy: 0.5068 - val_loss: 0.6938 - val_accuracy: 0.5085
Epoch 3/1000
40762/40762 [==============================] - 1s 30us/sample - loss: 0.6935 - accuracy: 0.5127 - val_loss: 0.6931 - val_accuracy: 0.5072
Epoch 4/1000
40762/40762 [==============================] - 1s 29us/sample - loss: 0.6929 - accuracy: 0.5177 - val_loss: 0.7008 - val_accuracy: 0.5076
Epoch 5/1000
40762/40762 [==============================] - 1s 30us/sample - loss: 0.6952 - accuracy: 0.5146 - val_loss: 0.6982 - val_accuracy: 0.5020
Epoch 6/1000
40762/40762 [==============================] - 1s 30us/sample - loss: 0.6945 - accuracy: 0.5139 - val_loss: 0.6942 - val_accuracy: 0.5121
Epo

40762/40762 [==============================] - 1s 30us/sample - loss: 0.6739 - accuracy: 0.5503 - val_loss: 0.7054 - val_accuracy: 0.5017
Epoch 31/1000
40762/40762 [==============================] - 1s 30us/sample - loss: 0.6698 - accuracy: 0.5554 - val_loss: 0.7085 - val_accuracy: 0.5040
Epoch 32/1000
40762/40762 [==============================] - 1s 30us/sample - loss: 0.6625 - accuracy: 0.5625 - val_loss: 0.7196 - val_accuracy: 0.4957
Epoch 33/1000
40762/40762 [==============================] - 1s 30us/sample - loss: 0.6582 - accuracy: 0.5640 - val_loss: 0.7258 - val_accuracy: 0.4994
Epoch 34/1000
40762/40762 [==============================] - 1s 30us/sample - loss: 0.6522 - accuracy: 0.5671 - val_loss: 0.7361 - val_accuracy: 0.5030
Epoch 35/1000
40762/40762 [==============================] - 1s 30us/sample - loss: 0.6480 - accuracy: 0.5729 - val_loss: 0.7262 - val_accuracy: 0.5051
Epoch 36/1000
40762/40762 [==============================] - 1s 30us/sample - loss: 0.6463 - accuracy:

40762/40762 [==============================] - 1s 30us/sample - loss: 0.6923 - accuracy: 0.5204 - val_loss: 0.6946 - val_accuracy: 0.5031
Epoch 14/1000
40762/40762 [==============================] - 1s 30us/sample - loss: 0.6917 - accuracy: 0.5234 - val_loss: 0.6991 - val_accuracy: 0.5012
Epoch 15/1000
40762/40762 [==============================] - 1s 30us/sample - loss: 0.6924 - accuracy: 0.5210 - val_loss: 0.6937 - val_accuracy: 0.5027
Epoch 16/1000
40762/40762 [==============================] - 1s 30us/sample - loss: 0.6918 - accuracy: 0.5226 - val_loss: 0.6951 - val_accuracy: 0.5043
Epoch 17/1000
40762/40762 [==============================] - 1s 30us/sample - loss: 0.6919 - accuracy: 0.5206 - val_loss: 0.6953 - val_accuracy: 0.5046
Epoch 18/1000
40762/40762 [==============================] - 1s 30us/sample - loss: 0.6918 - accuracy: 0.5218 - val_loss: 0.6946 - val_accuracy: 0.5038
Epoch 19/1000
40762/40762 [==============================] - 1s 30us/sample - loss: 0.6908 - accuracy:

40762/40762 [==============================] - 1s 30us/sample - loss: 0.6771 - accuracy: 0.5470 - val_loss: 0.7028 - val_accuracy: 0.4989
Epoch 32/1000
40762/40762 [==============================] - 1s 30us/sample - loss: 0.6747 - accuracy: 0.5503 - val_loss: 0.7082 - val_accuracy: 0.4975
Epoch 33/1000
40762/40762 [==============================] - 1s 30us/sample - loss: 0.6722 - accuracy: 0.5494 - val_loss: 0.7086 - val_accuracy: 0.4983
Epoch 34/1000
40762/40762 [==============================] - 1s 30us/sample - loss: 0.6682 - accuracy: 0.5547 - val_loss: 0.7109 - val_accuracy: 0.4989
Epoch 35/1000
40762/40762 [==============================] - 1s 30us/sample - loss: 0.6645 - accuracy: 0.5560 - val_loss: 0.7128 - val_accuracy: 0.5027
Epoch 36/1000
40762/40762 [==============================] - 1s 30us/sample - loss: 0.6626 - accuracy: 0.5593 - val_loss: 0.7220 - val_accuracy: 0.5058
Epoch 37/1000
40762/40762 [==============================] - 1s 30us/sample - loss: 0.6569 - accuracy:

Epoch 13/1000
40815/40815 [==============================] - 1s 29us/sample - loss: 0.6922 - accuracy: 0.5206 - val_loss: 0.6956 - val_accuracy: 0.5003
Epoch 14/1000
40815/40815 [==============================] - 1s 29us/sample - loss: 0.6937 - accuracy: 0.5225 - val_loss: 0.6949 - val_accuracy: 0.5052
Epoch 15/1000
40815/40815 [==============================] - 1s 29us/sample - loss: 0.6918 - accuracy: 0.5204 - val_loss: 0.6936 - val_accuracy: 0.5117
Epoch 16/1000
40815/40815 [==============================] - 1s 29us/sample - loss: 0.6916 - accuracy: 0.5238 - val_loss: 0.6982 - val_accuracy: 0.5031
Epoch 17/1000
40815/40815 [==============================] - 1s 29us/sample - loss: 0.6914 - accuracy: 0.5217 - val_loss: 0.6954 - val_accuracy: 0.5017
Epoch 18/1000
40815/40815 [==============================] - 1s 29us/sample - loss: 0.6915 - accuracy: 0.5228 - val_loss: 0.6936 - val_accuracy: 0.5154
Epoch 19/1000
40815/40815 [==============================] - 1s 29us/sample - loss: 0.69

In [31]:
print(f'Mean: of significance = {significance:.2f}', np.mean(epsilon_results, axis = 0))
print(f'Std of significance = {significance:.2f}: ', np.std(epsilon_results, axis = 0))


Mean: of significance = 0.32 [0.00064175 0.010218   0.108738  ]
Std of significance = 0.32:  [0.00025207 0.00261597 0.02780098]
